# MLflowテスト

`autolog()`  
MLflow の自動ログ機能を使用すると、1行のコードで、結果のモデル、モデルの作成に使用されたパラメーター、およびモデル スコアが自動的にログに記録される

`mlflow.log_metric()など`  
各トレーニング実行でログに記録されるメトリックをさらに制御する必要がある場合、  
またはテーブルやプロットなどの追加の成果物をログに記録する場合は、このノートブックで説明されているおよびAPIを使用できる

In [ ]:
import mlflow
import mlflow.sklearn
# import mlflow.lightgbm
import pandas as pd
import matplotlib.pyplot as plt

from numpy import savetxt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
mlflow.sklearn.autolog()
# mlflow.lightgbm.autolog()

In [ ]:
db = load_diabetes()
X = db.data
y = db.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
with mlflow.start_run():
  
  # Set the model parameters. 
  n_estimators = 100
  max_depth = 6
  max_features = 3
  
  # Create and train model.
  rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
  rf.fit(X_train, y_train)
  
  # Use the model to make predictions on the test dataset.
  predictions = rf.predict(X_test)
  
  # トレーニングパラメータ
  mlflow.log_param("num_trees", n_estimators)
  mlflow.log_param("maxdepth", max_depth)
  mlflow.log_param("max_feat", max_features)
  
  # Define a metric to use to evaluate the model.
  mse = mean_squared_error(y_test, predictions)
  
  # 数値メトリクス
  mlflow.log_metric("mse", mse)
  
  # モデル
  mlflow.sklearn.log_model(rf, "random-forest-model") 
  
  # Save the table of predicted values
  savetxt('predictions.csv', predictions, delimiter=',')
  
  # Log the saved table as an artifact
  # ファイル系
  mlflow.log_artifact("predictions.csv")
  
  # Convert the residuals to a pandas dataframe to take advantage of graphics capabilities
  df = pd.DataFrame(data = predictions - y_test)
  
  # Create a plot of residuals
  plt.plot(df)
  plt.xlabel("Observation")
  plt.ylabel("Residual")
  plt.title("Residuals")

  # Save the plot and log it as an artifact
  
  plt.savefig("residuals_plot.png")
  mlflow.log_artifact("residuals_plot.png") 